First, lets list available embedding models

In [1]:
from anaconda_ai import AnacondaAIClient

client = AnacondaAIClient()

embedding_models = [
    model.name
    for model in client.models.list()
    if model.trained_for == "sentence-similarity"
]
embedding_models

['BGE-Large-EN-V1.5', 'bge-small-en-v1.5', 'Nomic-Embed-Text-v1.5']

Now we can start and embedding server and create a vectordb table 

In [2]:
from anaconda_ai.integrations.langchain import AnacondaQuantizedModelEmbeddings

embeddings_model = AnacondaQuantizedModelEmbeddings(
    model_name="bge-small-en-v1.5_Q4_K_M.gguf"
)

Output()

✓ bge-small-en-v1.5_Q4_K_M.gguf (running)

In [3]:
from langchain_postgres import PGVector

vectordb = client.vector_db.create()

host = vectordb.host
port = vectordb.port
database = vectordb.database
user = vectordb.user
password = vectordb.password

connection = f"postgresql+psycopg://{user}:{password}@{host}:{port}/{database}"
collection_name = "test_collection"

vector_store = PGVector(
    embeddings=embeddings_model,
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)

Output()

✓ pg vector database started

In [4]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="wild birds gather at the lake",
        metadata={"id": 1, "location": "lake", "topic": "wildlife"},
    ),
    Document(
        page_content="fish can be spotted swimming in the lake",
        metadata={"id": 2, "location": "lake", "topic": "wildlife"},
    ),
    Document(
        page_content="fresh vegetables are sold at the farmers market",
        metadata={"id": 3, "location": "farmers market", "topic": "groceries"},
    ),
    Document(
        page_content="the farmers market features local honey products",
        metadata={"id": 4, "location": "farmers market", "topic": "groceries"},
    ),
    Document(
        page_content="the historical exhibit showcases ancient artifacts",
        metadata={"id": 5, "location": "heritage center", "topic": "history"},
    ),
    Document(
        page_content="a photography display is featured at the heritage center",
        metadata={"id": 6, "location": "heritage center", "topic": "history"},
    ),
    Document(
        page_content="a new bakery opened downtown on Park Avenue",
        metadata={"id": 7, "location": "Park Avenue", "topic": "dining"},
    ),
    Document(
        page_content="the science club meets at the research center",
        metadata={"id": 8, "location": "research center", "topic": "education"},
    ),
    Document(
        page_content="the research center hosts monthly astronomy nights",
        metadata={"id": 9, "location": "research center", "topic": "education"},
    ),
    Document(
        page_content="a yoga class for seniors is available at the wellness pavilion",
        metadata={"id": 10, "location": "wellness pavilion", "topic": "fitness"},
    ),
]

vector_store.add_documents(docs, ids=[doc.metadata["id"] for doc in docs])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
results = vector_store.similarity_search("animal", k=3)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* fish can be spotted swimming in the lake [{'id': 2, 'topic': 'wildlife', 'location': 'lake'}]
* wild birds gather at the lake [{'id': 1, 'topic': 'wildlife', 'location': 'lake'}]
* the historical exhibit showcases ancient artifacts [{'id': 5, 'topic': 'history', 'location': 'heritage center'}]
